## UCZENIE RĘKI ROBOTA PRZEPYCHANIA KRĄŻKA W OKREŚLONY PUNKT


In [1]:
import gym
import gym.wrappers as wr
from gym import Wrapper
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline


import torch
torch.manual_seed(0) 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical


from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
def reset(env):
    state = env.reset()
    rrr = env.env.env
    al = []
    if rrr.has_object:
            object_xpos = rrr.initial_gripper_xpos[:2]
            object_qpos = rrr.sim.data.get_joint_qpos('object0:joint')
            assert object_qpos.shape == (7,)
            object_qpos[:2] = object_xpos
            object_qpos[0] = 1.10
            object_qpos[1] = 0.75
            object_qpos[3] = 0
            object_qpos[-2] = 0.6
            rrr.sim.data.set_joint_qpos('object0:joint', object_qpos)
            al = object_qpos[0:3]

    env.env.env = rrr
    env.env.env.goal = np.array([1.3, 0.70, 0.42186031])
    state = np.concatenate(( np.array(state['observation']) ,np.array(al), np.array([1.3, 0.70, 0.42186031])))

    
    return state

def step(env,action):
    state, reward, done, a = env.step(tuple(action))
    reward = -1*((state ["desired_goal"][0] - state ["achieved_goal"][0])**2 +(state ["desired_goal"][1] - state ["achieved_goal"][1])**2 +(state ["desired_goal"][2] - state ["achieved_goal"][2])**2 )

    if a["is_success"] > 0:
        reward+=3000

    state = np.concatenate(( np.array(state['observation']) ,np.array(state["achieved_goal"]), np.array(state["desired_goal"])))
    return state, reward, done, a
    

In [3]:
# Przygotowanie środowiska 
env = gym.make('FetchSlide-v1')

env.env.reward_type="dense"

obs = env.reset()
rrr = env.env
al = []
if rrr.has_object:
        object_xpos = rrr.initial_gripper_xpos[:2]
        object_qpos = rrr.sim.data.get_joint_qpos('object0:joint')
        assert object_qpos.shape == (7,)
        object_qpos[:2] = object_xpos
        print(object_qpos)
        object_qpos[0] = 1.10
        object_qpos[1] = 0.75
        object_qpos[3] = 0
        object_qpos[-2] = 0.6
        rrr.sim.data.set_joint_qpos('object0:joint', object_qpos)
        al = object_qpos[0:3]

env.env = rrr
env.env.goal = np.array([1.8, 0.75, 0.42186031])

env = wr.Monitor(env, '/media/patryk/Inne_pliki/Pobrane/', force = True)

[ 0.99570627  0.74890684  0.42185313  0.70644302 -0.49749877  0.50338356
  0.00618184]


In [ ]:
class Agent(nn.Module):
    def __init__(self,env,input_size = 31,hidden_size = 310,output_size = 4):
        super(Agent, self).__init__()
        
        self.env = env
        self.i_s = input_size
        
        self.h_s = hidden_size
        self.o_s = output_size 
        
        self.fc1 = nn.Linear(self.i_s, self.h_s)
        self.fc2 = nn.Linear(self.h_s, self.o_s)
        
    def ustaw_wagi(self,wagi):
        h = self.h_s
        i = self.i_s
        o = self.o_s
        fc1_rozm = (i*h)+h
        
        fc1_W = torch.from_numpy(wagi[:i*h].reshape(i, h)) #Wagi 1 warstwa
        fc1_b = torch.from_numpy(wagi[i*h:fc1_rozm]) #Bias 1 warstwa
        fc2_W = torch.from_numpy(wagi[fc1_rozm:fc1_rozm+(h*o)].reshape(h, o)) #Wagi 2 warstwa
        fc2_b = torch.from_numpy(wagi[fc1_rozm+(h*o):]) #Bias 2 warstwa
        
        self.fc1.weight.data.copy_(fc1_W.view_as(self.fc1.weight.data))
        self.fc1.bias.data.copy_(fc1_b.view_as(self.fc1.bias.data))
        self.fc2.weight.data.copy_(fc2_W.view_as(self.fc2.weight.data))
        self.fc2.bias.data.copy_(fc2_b.view_as(self.fc2.bias.data))
        
    
    def get_wagi(self):
        return self.i_s*self.h_s+ self.h_s + self.h_s*self.o_s + self.o_s
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return x.cpu().data
        
    def evaluate(self, wagi, gamma=1.0, max_t=5000):
        self.ustaw_wagi(wagi)
        episode_return = 0.0
        state = reset(self.env)
    
        for t in range(max_t):
            state = torch.from_numpy(state).float().to(device)
            action = self.forward(state)
            state, reward, done, a = step(env, (tuple(action)))
            episode_return += reward * math.pow(gamma, t)
            if done:
                break
        return episode_return
    
agent = Agent(env)      
        

In [ ]:
def cem(N=5000, max_t=1000, gamma=1.0, print_for=10, populacja=30, ile_naj=0.25, sigma=0.3):
    n_elite=int(populacja*ile_naj)

    kolejka = deque(maxlen=100)
    wyniki = []
    najlepszy = sigma*np.random.randn(agent.get_wagi())

    for iteracja in range(1, N+1):
        populacja_nowych = [najlepszy + ((sigma**i+sigma)*np.random.randn(agent.get_wagi())) for i in range(populacja)]
        tab = np.array([agent.evaluate(wynikowa, gamma, max_t) for wynikowa in populacja_nowych])
        
        indeksy = tab.argsort()[-n_elite:]
        najlepsze_wagi = [populacja_nowych[i] for i in indeksy]
        najlepszy = np.array(najlepsze_wagi).mean(axis=0)

        wynik = agent.evaluate(najlepszy, gamma=1.0)
        kolejka.append(wynik)
        wyniki.append(wynik)
        
        torch.save(agent.state_dict(), 'checkpoint.pth')
        
        if iteracja % print_for == 0:
            print('Episod: {}\tŚredni wynik: {:.2f}'.format(iteracja, np.mean(kolejka)))

    return wyniki

scores = cem()

Episod: 10	Średni wynik: -1.61
Episod: 20	Średni wynik: 2698.95
Episod: 30	Średni wynik: 1799.13
Episod: 40	Średni wynik: 1349.23
Episod: 50	Średni wynik: 1079.28
Episod: 60	Średni wynik: 899.32
Episod: 70	Średni wynik: 770.78
Episod: 80	Średni wynik: 674.37
Episod: 90	Średni wynik: 899.39
Episod: 100	Średni wynik: 809.40
Episod: 110	Średni wynik: 1649.51
Episod: 120	Średni wynik: 1379.34
Episod: 130	Średni wynik: 1679.02
Episod: 140	Średni wynik: 2818.59
Episod: 150	Średni wynik: 3118.28
Episod: 160	Średni wynik: 3328.15
Episod: 170	Średni wynik: 3328.15
Episod: 180	Średni wynik: 3328.15
Episod: 190	Średni wynik: 3058.14
Episod: 200	Średni wynik: 3058.15
Episod: 210	Średni wynik: 3148.15
Episod: 220	Średni wynik: 2878.32
Episod: 230	Średni wynik: 3058.64
Episod: 240	Średni wynik: 1919.06
Episod: 250	Średni wynik: 1619.38
Episod: 260	Średni wynik: 1829.51
Episod: 270	Średni wynik: 1829.51
Episod: 280	Średni wynik: 1829.51
Episod: 290	Średni wynik: 1829.51
Episod: 300	Średni wynik: 1829

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('WYNIK')
plt.xlabel('EPISOD')
plt.show()

In [ ]:

img = plt.imshow(env.render(mode='rgb_array'))
state = reset(env)
while True:
    state = torch.from_numpy(state).float().to(device)
    
    with torch.no_grad():
        action = agent(state)
    img.set_data(env.render(mode='rgb_array')) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    state, reward, done, a = step(env,tuple(action))
    if done:
        break

env.close()